# Multiple Linear Regression

This chapter focuses on determining the strength of relationships between a collection of (assumed) linearly independent variables $x_{i}$ and a real continuous dependent variable $y$,

\begin{equation}
    y = b_0 + b_1 x_1 + b_2 x_2 + \ldots + b_n x_n,
\end{equation}

using supervised machine learning. This is known as multiple linear regression.

First let's import the libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Read in the data, and take a look at the column titles and first few rows.

In [2]:
dataset = pd.read_csv('50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


We see that the dependent variable of interest, Profit, is in column `4`, and assume the rest of the data form independant variables with some impact on Profit.

In [3]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

## Encode Categorical Independant Variables

We need to encode the State column using numbers that the machine learning algorithm can understand.

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])
onehotencoder = OneHotEncoder(categorical_features = [3])
X = onehotencoder.fit_transform(X).toarray()

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


## Avoiding the Dummy Variable Trap

Next we need to avoid the dummy variable trap. The encoded categorical variables are not linearly independent,

\begin{equation}
    D_1 = 1 - D_2 - D_3.
\end{equation}


I.e. there are $n - 1$ degrees of freedom for $n$ states. We need to remove one of the encoded variables. This step is optional, as the regression library will take avoid the Dummy Variable for us, however, for some libraries, this is not the case. It does no harm to deal with it ourselves.

In [5]:
X = X[:, 1:]

## Split the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Fitting Multiple Linear Regression to the Training set

In [7]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

## Predicting the Test set results

In [8]:
y_pred = regressor.predict(X_test)

## Building the optimal model using Backward Elimination

Above, we fitted our model using all available input variables. However, it's possible that some of these have a negiligible impact on Profit. Including these provides no improvement to predictability and can have a negative impact by unnessarily exposing our model to overfitting. Also, they clutter the picture, and distract from the underlying processes. Let's perform backwards elimination and demand that a significance level of 0.05 for a predictor to remain in the model at each iteration.

In [9]:
import statsmodels.formula.api as sm

Next we need to incorporate the overall constant $b_{0}$ in our model by including $x_0 \equiv 1$.

In [10]:
X = np.append(arr=np.ones((X.shape[0], 1)).astype(int), values=X, axis=1)

First, we fit the model with all possible predictors.

Some definitions:
* OLS: Ordinary Least Squares
* `endog`: endogenous response variable - the dependent variable (y)
* `exog`: exogenous variables designates variables that appear in the model, but are not explained by that model, i.e. the independent input variables (x)
* endogenous: (informal) caused by factors within the system
* exogenous: (informal) caused by factors outside the system

In [12]:
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()

Print the summary to look at the p-values for each independent variable.

In [13]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 15 May 2019   Prob (F-statistic):           1.34e-27
Time:                        10:50:45   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x2           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Let's remove $x_2$, i.e. the second dummy variables for state.

In [14]:
X_opt = X[:, [0, 1, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Wed, 15 May 2019   Prob (F-statistic):           8.49e-29
Time:                        11:40:23   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
x1           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
X_opt = X[:, [0, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Wed, 15 May 2019   Prob (F-statistic):           4.53e-30
Time:                        11:44:46   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
X_opt = X[:, [0, 3, 5]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Wed, 15 May 2019   Prob (F-statistic):           2.16e-31
Time:                        11:45:34   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
X_opt = X[:, [0, 3]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Wed, 15 May 2019   Prob (F-statistic):           3.50e-32
Time:                        11:46:24   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Automatic Backward Elimination

### Backward Elimination with p-values only


We can automate the above steps with the following code.

In [ ]:
import statsmodels.formula.api as sm
def backward_elimination(x, sl):
    n_vars = len(x[0])
    for i in range(0, n_vars):
        regressor_OLS = sm.OLS(y, x).fit()
        max_var = max(regressor_OLS.pvalues).astype(float)
        if max_var > sl:
            for j in range(0, n_vars - i):
                if (regressor_OLS.pvalues[j].astype(float) == max_var):
                    x = np.delete(x, j, 1)
    print(regressor_OLS.summary())
    return x
 
sl = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backward_elimination(X_opt, sl)

### Backward Elimination with p-values and Adjusted R-squared

The coefficient of determination or $R^2$ tells you how much variation is explained by your model. The calculation is one minus the sum of squares of residuals, divided by the total sum of squares,

\begin{equation}
    R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2},
\end{equation}

where $\hat{y}_i$ is the predicted value corresponding to the actual value $y_i$, and $\bar{y}$ is the mean of the $\{y_i\}$.

So $0.1$ $R^2$ means that the model explains $10\%$ of the variation within the data, i.e. the greater $R^2$, the better the model.
There is no established relationship or association between p-value and $R^2$, it is all data contextual. $R^2$ is just a measure of the goodness-of-fit of the regression; a purely "geometrical" quantity. The p-value is a measure of evidence against the hypothesis that the regression coefficient is zero. 

Thus there are four scenarios:
1. Low $R^2$ and low p-value (p-value $\leq 0.05$)
2. Low $R^2$ and high p-value (p-value $> 0.05$)
3. High $R^2$ and low p-value
4. High $R^2$ and high p-value

Interpretation:
1. The model doesn't explain much of the variation in the data but it is significant (better than nothing)
2. The model doesn't explain much of the variation in the data and it is not significant (worst case scenario)
3. The model explains a lot of variation within the data and is significant (best case scenario)
4. The model explains a lot of variation within the data but is not significant (model is worthless)

#### Problems with R-squared

$R^2$ has some problems:

1. *More is always better*: every time a predictor is added to a model, the $R^2$ increases; regardless of predicting power, it never decreases. Consequently, a model with more terms may appear to have a better fit simply because it has more terms.

2. *Overfitting*: if a model has too many predictors (and/or higher order polynomials), it begins to model the random noise in the data, i.e. it leads to overfitting. This produces misleadingly high $R^2$ values and poorer predictions in the test data.

#### Adjusted R-squared

The adjusted $R^2$ is a modified version of $R^2$ that has been adjusted for the number of predictors in the model. The adjusted $R^2$ increases only if the new term improves the model more than would be expected by chance. It decreases when a predictor improves the model by less than expected by chance. The adjusted $R^2$ can be negative, but it’s usually not.  It is always lower than the $R^2$. The formula for adjusted $R^2$ is

\begin{equation}
    R^2_{adj} = 1 - (1 - R^2) \frac{n - 1}{n - k - 1},
\end{equation}

where $n$ is the number of points in the data sample, and $k$ is the number of independent variables.
We can automate a backwards elimination using both the p-value and $R^2$ using the formula below.

In [27]:
import statsmodels.formula.api as sm
def backward_elimination_adjR(x, SL):
    n_vars = len(x[0])
    temp = np.zeros((50,6)).astype(int)
    for i in range(0, n_vars):
        regressor_OLS = sm.OLS(y, x).fit()
        max_var = max(regressor_OLS.pvalues).astype(float)
        adjR_before = regressor_OLS.rsquared_adj.astype(float)
        if max_var > SL:
            for j in range(0, n_vars - i):
                if (regressor_OLS.pvalues[j].astype(float) == max_var):
                    temp[:,j] = x[:, j]
                    x = np.delete(x, j, 1)
                    tmp_regressor = sm.OLS(y, x).fit()
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
                    if (adjR_before >= adjR_after):
                        x_rollback = np.hstack((x, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (regressor_OLS.summary())
                        return x_rollback
                    else:
                        continue
    regressor_OLS.summary()
    return x
 
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backward_elimination_adjR(X_opt, SL)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Wed, 15 May 2019   Prob (F-statistic):           2.16e-31
Time:                        12:51:44   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.0

Neither $R^2$ nor $R^2_{adj}$ can determine whether the coefficient estimates and predictions are biased. This must be assessed by looking in detail at the residual plots.